In [1]:
import os
import argparse
import numpy as np
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset,DataLoader

import torchvision.utils as utils
import torchvision.transforms as transforms
import torchvision.models as models


import copy
import glob
import torch.utils.data as udata
import h5py
import cv2
import random
import pywt
import matplotlib.pyplot as plt
from PIL import Image
import math
from skimage.measure.simple_metrics import compare_psnr
import scipy.io as sio
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #Anaconda内的一个.dll和pytorch的一个libiomp5md.dll冲突了，仅需要允许备份存在即可
plt.rcParams['font.sans-serif'] = [u'SimHei'] #正常输出中文字符 黑体
plt.rcParams['axes.unicode_minus'] = False #正常输出正负号


In [2]:
#多设备支持策略
use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor #可以用gpu的话，数据类型是torch.cuda.FloatTensor 否则使用torch.FloatTensor
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

path_save = './style_attack' #图片保存路径

# desired size of the output image
# imsize = 128 # use small size if no gpu 图片大小
imsize = 512

In [3]:
#一些相关函数
loader = transforms.Compose([ 
     transforms.Resize([imsize, imsize]),  # scale imported image 变换图片为对应大小
     transforms.ToTensor()])  # transform it into a torch tensor  转化为张量

def imnoise(img, sigma): #添加随机噪声
    return skimage.util.random_noise(img / 255, mode='gaussian', seed=None, var=(sigma/255.0)**2) * 255
#      随机噪声处理是对[0,1]之间的数据进行添加的，即黑白图像，所以输入图像需要/255，最后处理结束*255又变回彩色图像；噪声模型采用高斯加性模型，
#  不设置随机种子；高斯分布的标准差希望是sigma

def image_loader(image_name): #数据加载 读进图片进行加载
    image = Image.open(image_name)  #打开图片
    image = Variable(loader(image)) #图片从张量变成变量，Variable是网络参数优化的必须数据类型，可以对梯度进行计算保存
    # fake batch dimension required to fit network's input dimensions
    image = image.unsqueeze(0) #在第一维度之后再加一个维度 ？不知道为啥
    return image

unloader = transforms.ToPILImage()  # reconvert into PIL image 变回图片


def image_unloader(tensor):
    image = tensor.clone().to(device)  # we clone the tensor to not do changes on it 把.cpu()去掉就行
    image = image.view(3, imsize, imsize)  # remove the fake batch dimension  图片变为3维 128*128的图片
    image = unloader(image) #变回图片
    return image

def pil2tensor(pil_img):  #图片转换为张量
    image = Variable(loader(pil_img)) #loader转化为128*128,Variable,则为128*128
    # fake batch dimension required to fit network's input dimensions
    image = image.unsqueeze(0) #网络输入维度 维度1*128*128 ？或3*128*128
    return image
    
def imshow(tensor, title=None): #图片展示
    image = tensor.clone().to(device) # we clone the tensor to not do changes on it#感觉这两行没啥毕必要
    image = image.view(3, imsize, imsize)  # remove the fake batch dimension 
    image = unloader(image)
    plt.imshow(image) #显示图片
    if title is not None:
        plt.title(title)
    plt.show()# pause a bit so that plots are updated
    
    
def get_pil_image(image_name):
    img = image_loader(image_name)
    img = image_unloader(img)
    return img

def pil_image_croper(pil_img, crop=(0, 0, 128, 128)): #图片裁剪
    return pil_img.crop(crop) #图片裁剪 仅仅裁剪左上角的一个128*128的正方形

def pil_image_croppers(pil_img, strider = 128): #把图片从上到下划分为128*128的小方形，按照从上到下再从左边到右边加入列表 5
    x1 = 0
    y1 = 0
    x2 = 128
    y2 = 128
    lst = []
    while x2 < 256:
        y1 = 0
        y2 = 128
        while y2 < 256:
            print((x1, y1, x2, y2))
            img = pil_image_croper(pil_img, (x1, y1, x2, y2))
            lst.append(img)
            y1 += strider
            y2 += strider
        x1 += strider
        x2 += strider
    return lst 

def pil_image_add_noise(img, noiseL=0.005):  #添加噪声 6
    img = img.squeeze()
    channel, width, height = img.size()
    noise = torch.FloatTensor(img.shape).normal_(mean=0, std=noiseL/255)
    img_noise = img + noise
    return image_unloader(img_noise),noiseL

def pil_image_resize(img, mimsize=128): #图片修改尺寸为128*128 3
    mloader = transforms.Compose([
    transforms.Resize([mimsize, mimsize]),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor
    image = Variable(mloader(img))
    # fake batch dimension required to fit network's input dimensions
    image = image.unsqueeze(0)
    image = image.clone().to(device)  # we clone the tensor to not do changes on it
    image = image.view(3, mimsize, mimsize)  # remove the fake batch dimension
    image = unloader(image)
    image = pil_image_croper(image, (0, 0, 128, 128))
    return image

def pil_image_rotate(img, degree = 45): #旋转45度 4
    im_rotate = img.rotate(degree) 
    #img_totate = pil_image_croper(im_rotate, (128,128,256,256)) #裁剪128*128的大小
    return im_rotate

def pil_image_compress_save(img, outfile, quality=80): #1
    img.save(outfile, quality=quality) #保存图片到out,file，分辨率为80dpi 每英寸80个点
    
def pil_image_qt(img, ratio=128): #2 jepg攻擊
    # image=data.coffee()
    image = pil2tensor(img).squeeze().numpy() * 255
    image = image.transpose([1,2,0])
    ratio=128  # 设置量化比率
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            for k in range(image.shape[2]):
                image[i][j][k]=int(image[i][j][k]/ratio)*ratio
    np.clip(image, 0, 255)
    image = image.transpose([2,0,1])
    # image = image/255
    image = torch.from_numpy(image).unsqueeze(0)
    image = image_unloader(image)
    return image

In [4]:
# noise 
# imsize = 128
target_img = './style/1.png'
for i in range(1,100):
    img = get_pil_image(target_img)
    img = pil2tensor(img)
#     print(img.type())
#     print(img.shape)
    img_name = '1_noise_'+str(i)+'.jpg'
    img,noise = pil_image_add_noise(img,i)
    img.save(os.path.join(path_save,img_name))



In [5]:
# # quality 
imsize = 128
target_img = './style/1.png'  #a
for i in range(1,100):
    img = get_pil_image(target_img)
    img_name = '1_quality_'+str(i)+'.jpg'
    img = pil_image_qt(img,i)
    img.save(os.path.join(path_save,img_name))

In [6]:
# rotate 
imsize = 512
target_img = './style/1.png'  #
for i in range(0,300,1):
    img = get_pil_image(target_img)
    img_name = '1_rotate_'+str(i)+'.jpg'
    img = pil_image_rotate(img,i)
    img.save(os.path.join(path_save,img_name))

In [7]:
#crop 
imsize = 512
target_img = './style/1.png' 
strider = 50
image = get_pil_image(target_img)
x1 = 0
y1 = 0
x2 = 128
y2 = 128
for i in range(0,128,50):
    for j in range(0,128,50):
        #print(i,j,i+128,j+128)
        img = pil_image_croper(image, (i, j, i+128, j+128))
        img_name = '1_crop_'+str(i)+'_'+str(j)+'.jpg'
        img.save(os.path.join(path_save,img_name))

In [8]:
#resize 
imsize = 512
target_img = './style/1.png' #
for i in range(128,256,4):
    img = get_pil_image(target_img)
    img_name = '1_resize_'+str(i)+'.jpg'
    img = pil_image_resize(img,i)
    img.save(os.path.join(path_save,img_name))

In [9]:
#试一下，产生更多的攻击图片 compress a
# imsize = 512
for i in range(1,100):
    target_img = './style/1.png'
    img = get_pil_image(target_img)
    img_name = '1_compress_'+str(i)+'.jpg'
    pil_image_compress_save(img,os.path.join(path_save,img_name),i)